In [77]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [2]:
X_train = pd.read_csv('task2_k49am2lqi/train_features.csv')
y_train = pd.read_csv('task2_k49am2lqi/train_labels.csv')

In [3]:
X_test = pd.read_csv('task2_k49am2lqi/test_features.csv')

In [12]:
X_test.head(14)

,pid,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,0.0,39.0,-1.000000,7.518182,3.363636,-1.000,16.363636,1.754545,2.636364,-1.300000,...,20.727273,81.818182,-1.0,18.181818,5.172727,61.454545,4.381818,-1.000000,93.545455,1.474000
12,10001.0,62.0,-1.000000,-1.000000,-1.000000,-1.000,14.900000,-1.000000,-1.000000,-1.000000,...,-1.000000,85.636364,-1.0,-1.000000,-1.000000,101.272727,-1.000000,-1.000000,91.181818,-1.000000
24,10003.0,72.0,-1.000000,-1.000000,-1.000000,0.402,36.909091,-1.000000,-1.000000,-1.000000,...,-1.000000,99.636364,-1.0,-1.000000,-1.000000,91.000000,-1.000000,-1.000000,124.909091,1.492000
36,10004.0,44.0,-1.000000,-1.000000,1.666667,-1.000,13.181818,2.116667,-1.000000,-1.000000,...,20.500000,95.727273,-1.0,-1.000000,6.250000,87.636364,0.033333,-1.000000,94.727273,-1.000000
48,10005.0,88.0,-1.000000,9.975000,10.500000,-1.000,5.142857,2.200000,6.250000,-1.000000,...,-1.000000,95.909091,-1.0,27.000000,7.400000,54.818182,-1.000000,-1.000000,118.272727,-1.000000
60,10008.0,59.0,-1.000000,3.085714,11.142857,1.360,12.166667,3.657143,7.428571,2.800000,...,22.571429,98.250000,-1.0,36.166667,10.628571,71.250000,-1.000000,-1.000000,154.000000,6.036000
72,10011.0,45.0,-1.000000,-1.000000,3.500000,-1.000,4.000000,4.500000,-1.000000,-1.000000,...,-1.000000,99.545455,-1.0,-1.000000,13.600000,66.181818,-1.000000,-1.000000,149.090909,-1.000000
84,10017.0,36.0,-1.000000,2.950000,1.125000,-1.000,10.000000,3.125000,-1.000000,-1.000000,...,9.000000,89.636364,-1.0,-1.000000,9.525000,96.909091,0.050000,-1.000000,147.909091,-1.000000
96,10018.0,65.0,15.363636,-1.000000,2.300000,-1.000,9.500000,0.870000,-1.000000,-1.000000,...,-1.000000,94.545455,-1.0,-1.000000,2.950000,109.636364,-1.000000,-1.000000,106.000000,0.667273
108,10019.0,84.0,-1.000000,-1.000000,2.090909,-1.000,36.363636,0.854545,2.090909,-0.545455,...,-1.000000,96.363636,-1.0,30.272727,7.981818,67.727273,-1.000000,-1.000000,116.181818,2.679091


In [5]:
def process_data(X):
    X = X.fillna(0)
    X_new = pd.DataFrame(columns=X.columns).drop('Time', axis=1)
    length = X.shape[1] - 3

    for i in range(0, X.shape[0], 12):
        new_f = np.zeros(length + 2)
        for j in range(length):
            temp = 0
            counts = 0
            for k in range(12):
                temp += X.iloc[i + k ,3 + j]
                if temp != 0:
                    counts += 1
        
            if counts == 0:
                new_f[j + 2] = -1 #No valid measurement
            else:
                new_f[j + 2] = temp/counts
    
        new_f[0] = X.iloc[i, 0]
        new_f[1] = X.iloc[i, 2]
    
        X_new.loc[i] = new_f
    
    return X_new

In [6]:
X_train = process_data(X_train)

In [7]:
X_test = process_data(X_test)

In [107]:
X_train

,pid,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1.0,34.0,-1.000000,-1.000000,3.000000,-1.000,24.500000,2.141667,6.333333,-0.333333,...,-1.000000,100.000000,-1.0,28.000000,11.600000,77.083333,-1.000000,-1.000000,114.500000,4.299167
12,10.0,71.0,-1.000000,3.971429,1.714286,-1.000,9.000000,2.085714,-1.000000,-1.000000,...,9.714286,98.000000,-1.0,-1.000000,6.014286,78.818182,0.185714,0.001429,132.909091,-1.000000
24,100.0,68.0,-1.000000,1.900000,3.818182,-1.000,12.083333,2.272727,4.909091,-1.000000,...,-1.000000,96.500000,-1.0,18.363636,6.690909,109.083333,-1.000000,-1.000000,107.250000,-1.000000
36,1000.0,79.0,31.863636,-1.000000,11.000000,0.771,36.818182,4.600000,-1.000000,-1.000000,...,-1.000000,98.818182,-1.0,-1.000000,13.650000,86.363636,-1.000000,-1.000000,141.909091,0.912500
48,10000.0,76.0,-1.000000,5.190909,11.000000,-1.000,13.363636,1.945455,4.636364,0.666667,...,-1.000000,98.545455,-1.0,18.818182,5.509091,77.090909,-1.000000,-1.000000,123.000000,2.687273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227880,9993.0,80.0,-1.000000,-1.000000,2.454545,-1.000,15.888889,1.518182,-1.000000,-1.000000,...,11.818182,99.272727,-1.0,-1.000000,4.754545,112.818182,0.100000,1.941818,99.636364,-1.000000
227892,9995.0,73.0,-1.000000,6.937500,6.250000,-1.000,9.000000,1.400000,3.625000,-1.000000,...,-1.000000,94.545455,-1.0,11.125000,4.450000,62.636364,-1.000000,-1.000000,180.181818,-1.000000
227904,9996.0,53.0,-1.000000,-1.000000,-1.000000,-1.000,15.583333,1.800000,-1.000000,0.000000,...,-1.000000,99.583333,-1.0,-1.000000,7.508333,98.500000,-1.000000,-1.000000,99.250000,2.467500
227916,9998.0,89.0,-1.000000,3.154545,1.181818,-1.000,36.636364,0.763636,-1.000000,-1.000000,...,-1.000000,100.000000,-1.0,-1.000000,2.327273,97.454545,-1.000000,0.376250,131.545455,-1.000000


In [119]:
count = X_train[X_train < 0].count()
count

pid                     0
Age                     0
EtCO2               17673
PTT                 11179
BUN                  5029
Lactate             14123
Temp                  443
Hgb                  5020
HCO3                11158
BaseExcess          17069
RRate                 152
Fibrinogen          17281
Phosphate            9666
WBC                  5654
Creatinine           5268
PaCO2               11223
AST                 14199
FiO2                11318
Platelets            5454
SaO2                13940
Glucose              2665
ABPm                  107
Magnesium            6669
Potassium            3998
ABPd                 3882
Calcium              6729
Alkalinephos        14251
SpO2                   22
Bilirubin_direct    18372
Chloride            10644
Hct                  4356
Heartrate               7
Bilirubin_total     14284
TroponinI           16203
ABPs                  398
pH                  11001
dtype: int64

In [116]:
h = X_train.columns

pid


In [120]:
drop = []
for i in range(len(count)):
    if count[i] > 0.6*X_train.shape[0]:
        drop.append(h[i])

In [121]:
print(drop)

['EtCO2', 'Lactate', 'BaseExcess', 'Fibrinogen', 'AST', 'SaO2', 'Alkalinephos', 'Bilirubin_direct', 'Bilirubin_total', 'TroponinI']


In [122]:
X_train = X_train.drop(drop, axis=1)
X_test = X_test.drop(drop, axis=1)

In [123]:
X_train

,pid,Age,PTT,BUN,Temp,Hgb,HCO3,RRate,Phosphate,WBC,...,Magnesium,Potassium,ABPd,Calcium,SpO2,Chloride,Hct,Heartrate,ABPs,pH
0,1.0,34.0,-1.000000,3.000000,24.500000,2.141667,6.333333,17.000000,1.150000,1.308333,...,0.750000,1.000000,50.250000,1.900000,100.000000,28.000000,11.600000,77.083333,114.500000,4.299167
12,10.0,71.0,3.971429,1.714286,9.000000,2.085714,-1.000000,18.090909,0.357143,1.642857,...,0.214286,0.457143,83.272727,1.228571,98.000000,-1.000000,6.014286,78.818182,132.909091,-1.000000
24,100.0,68.0,1.900000,3.818182,12.083333,2.272727,4.909091,14.833333,0.318182,2.272727,...,0.309091,0.654545,62.833333,0.818182,96.500000,18.363636,6.690909,109.083333,107.250000,-1.000000
36,1000.0,79.0,-1.000000,11.000000,36.818182,4.600000,-1.000000,12.000000,0.950000,9.800000,...,1.000000,1.190000,62.818182,1.039000,98.818182,-1.000000,13.650000,86.363636,141.909091,0.912500
48,10000.0,76.0,5.190909,11.000000,13.363636,1.945455,4.636364,12.090909,-1.000000,1.409091,...,0.127273,0.709091,48.227273,-1.000000,98.545455,18.818182,5.509091,77.090909,123.000000,2.687273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227880,9993.0,80.0,-1.000000,2.454545,15.888889,1.518182,-1.000000,15.300000,-1.000000,1.645455,...,0.281818,0.727273,70.818182,1.454545,99.272727,-1.000000,4.754545,112.818182,99.636364,-1.000000
227892,9995.0,73.0,6.937500,6.250000,9.000000,1.400000,3.625000,19.545455,0.475000,1.700000,...,0.300000,0.612500,-1.000000,1.125000,94.545455,11.125000,4.450000,62.636364,180.181818,-1.000000
227904,9996.0,53.0,-1.000000,-1.000000,15.583333,1.800000,-1.000000,16.500000,-1.000000,-1.000000,...,0.450000,1.075000,55.958333,-1.000000,99.583333,-1.000000,7.508333,98.500000,99.250000,2.467500
227916,9998.0,89.0,3.154545,1.181818,36.636364,0.763636,-1.000000,20.454545,-1.000000,1.818182,...,-1.000000,0.363636,54.681818,0.681818,100.000000,-1.000000,2.327273,97.454545,131.545455,-1.000000


In [124]:
header = ['pid', 'LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2','LABEL_Sepsis','LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

In [125]:
#Task1

t1_labels = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
t1_train = np.array(y_train[t1_labels])



In [126]:
final = np.zeros((X_test.shape[0], len(header)))
final[:, 0] = X_test.values[:, 0]
for i in range(1, t1_train.shape[1] + 1):
    print(t1_labels[i - 1])
    model = LogisticRegression()
    model.fit(X_train.values, t1_train[:,i - 1])
    prediction = model.predict_proba(X_test)
    final[:, i] = prediction[:,1]

LABEL_BaseExcess


/home/rwanner/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LABEL_Fibrinogen


/home/rwanner/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LABEL_AST


...

/home/rwanner/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LABEL_Bilirubin_direct


/home/rwanner/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LABEL_EtCO2


/home/rwanner/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [127]:
#Task2
t2_labels = ['LABEL_Sepsis']
t2_train = np.array(y_train[t2_labels])

In [128]:
model = LogisticRegression()
model.fit(X_train.values, t2_train)
prediction = model.predict_proba(X_test)

/home/rwanner/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/rwanner/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [129]:
final[:, len(t1_labels) + 1] = prediction[:,1]

In [130]:
df = pd.DataFrame(final, columns=header)
df.head()

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,0.0,0.444481,0.172403,0.266849,0.267934,0.271004,0.337078,0.090714,0.367125,0.088224,0.253096,0.131727,0.0,0.0,0.0,0.0
1,10001.0,0.211472,0.073074,0.354441,0.351105,0.345481,0.169853,0.184221,0.180615,0.046497,0.030093,0.068708,0.0,0.0,0.0,0.0
2,10003.0,0.067447,0.063426,0.172555,0.201432,0.195078,0.211944,0.088577,0.335371,0.031247,0.077241,0.022013,0.0,0.0,0.0,0.0
3,10004.0,0.149878,0.085007,0.301267,0.305284,0.302693,0.165381,0.143197,0.153995,0.059249,0.135807,0.076113,0.0,0.0,0.0,0.0
4,10005.0,0.117924,0.029859,0.137723,0.126363,0.148555,0.089460,0.019988,0.083746,0.006708,0.004728,0.034185,0.0,0.0,0.0,0.0


In [131]:
#Task3
t3_labels = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
t3_train = np.array(y_train[t3_labels])

In [132]:
offset = len(t1_labels) + len(t2_labels) + 1
for i in range(t3_train.shape[1]):
    print(t3_labels[i])
    model = Ridge(alpha=10)
    model.fit(X_train.values, t3_train[:,i])
    prediction = model.predict(X_test)
    final[:, offset + i] = prediction

LABEL_RRate
LABEL_ABPm
LABEL_SpO2
LABEL_Heartrate


In [133]:
df = pd.DataFrame(final, columns=header)
df.head()

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,0.0,0.444481,0.172403,0.266849,0.267934,0.271004,0.337078,0.090714,0.367125,0.088224,0.253096,0.131727,15.711420,81.102313,97.798790,79.435710
1,10001.0,0.211472,0.073074,0.354441,0.351105,0.345481,0.169853,0.184221,0.180615,0.046497,0.030093,0.068708,18.226165,82.881003,96.717139,100.279994
2,10003.0,0.067447,0.063426,0.172555,0.201432,0.195078,0.211944,0.088577,0.335371,0.031247,0.077241,0.022013,19.344032,79.008671,97.074020,88.389631
3,10004.0,0.149878,0.085007,0.301267,0.305284,0.302693,0.165381,0.143197,0.153995,0.059249,0.135807,0.076113,15.821662,80.079211,97.592087,90.638697
4,10005.0,0.117924,0.029859,0.137723,0.126363,0.148555,0.089460,0.019988,0.083746,0.006708,0.004728,0.034185,19.236660,74.636925,97.019990,63.022030


In [134]:
df.to_csv('submission.zip', index=False, float_format='%.3f', compression = 'zip')